In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
%reload_ext autoreload

Import libraries and modules

In [61]:
from nn import nn, preprocess, io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error

In [62]:
#Load data
pos = io.read_text_file("./data/rap1-lieb-positives.txt")
neg = io.read_fasta_file("./data/yeast-upstream-1k-negative.fa")
print(len(pos))
print(len(neg))

137
3163


In [63]:
len(pos[50]),len(neg[50])

(17, 1000)

Length of positive sequences are 17bp and length of negative are 1000bp. Shorten the negative sequences to length of positive sequences and oversample positive sequences to balance classes. There are way fewer positive sequences than negative sequences

In [64]:
#Process negative sequences. Break up 1000bp sequence into 17bp chunks that are nonoverlapping

short_neg = []
target_length = len(pos[50])

for seq in neg:
    for i in range (0, len(seq), target_length):
        sub_neg_seq = seq[i:i+target_length]
        short_neg.append(sub_neg_seq)

In [65]:
len(short_neg)

186459

In [66]:
short_neg[:10]

['CTTCATGTCAGCCTGCA',
 'CTTCTGGGTCGTTGAAG',
 'TTTCTACCGATCAAACG',
 'CTTAGCGTCGAAAACGG',
 'TATTCGAAGGATTCATA',
 'GCAGCTTGATTCTTAGC',
 'AGCATCACCAATCAATC',
 'TTTCAGTGTCAGTGAAA',
 'GCGACAAAAGATGGAGT',
 'GGTTCTGTTACCTTGAT']

Combine sequences and label

In [67]:
all_seq = pos + short_neg
labels = [True] * len(pos) + [False] * len(short_neg)
print(len(all_seq))
print(all_seq[:10])
print(len(labels))
print(labels[:10])

186596
['ACATCCGTGCACCTCCG', 'ACACCCAGACATCGGGC', 'CCACCCGTACCCATGAC', 'GCACCCATACATTACAT', 'ACATCCATACACCCTCT', 'ACACCCTTACACTTTTA', 'GCATCCGTGCCTCCCAC', 'AAACCCATGCACAGTGA', 'ACATCCGTGCACCATTT', 'ACACCCATACATACGGA']
186596
[True, True, True, True, True, True, True, True, True, True]


Class balancing using preprocess.sample_seqs function

In [68]:
sampled_seqs, sampled_labels = preprocess.sample_seqs(all_seq, labels)

In [69]:
print(sampled_seqs[:5])
print(len(sampled_seqs))
print(sampled_labels[:5])
print(len(sampled_labels))

['ACATCCGTGCACCTCCG', 'ACACCCAGACATCGGGC', 'CCACCCGTACCCATGAC', 'GCACCCATACATTACAT', 'ACATCCATACACCCTCT']
186596
[True, True, True, True, True]
186596


One-hot encode sequences

In [70]:
X = preprocess.one_hot_encode_seqs(sampled_seqs)
y = np.array(sampled_labels, dtype=object)
print(X.shape)
print(y.shape)

(186596,)
(186596,)


/Users/tlou/Documents/2023/bmi203/final-nn/nn/preprocess.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  one_hot_encodings.append(np.array(base_one_hot).flatten())


Split into train and test sets

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [72]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(149276,)
(37320,)
(149276,)
(37320,)


Instantiate nn model and define parameters